<p style="font-weight:bold;"> <span style="font-size: 36px"> IFRS 17 Methodology </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Business Logic with Scopes  </span> </p>

In this notebook the focus is on the calculation of actuals values. 

# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "2ImportScope-PresentValue"

# Actual Values

We consider 4 types of Actual values, which are distinguished through their [Estimate Type](../DataModel/DataStructure#estimate-type):
- [Actuals](#actuals) (A)
- [Advance Actuals](#advance-actuals) (AA)
- [Overdue Actuals](#overdue-actuals) (OA)
- [Deferrable Actuals](#deferrable-actuals) (DA)

with the Estimate Type's system name shown between parenthesis above.

The following simplified AoC Chain applies for Advance and Overdue Actuals:
<br>&emsp;BOP
<br>&emsp;Release
<br>&emsp;Write-off
<br>&emsp;EOP

## Actual Base

The Actual Base sets values for actual, advance, and overdue as follows:

$$
\text{Actual Base} (\text{AoC step}) = \left\{
\begin{array}{rl}
0 ~, & \text{if AoC Step's AoC Type = AM} \\
\text{Actual Base}(\rm{BOP}) + \text{Actual Base}(\rm{CF}) + \text{Actual Base}(\rm{WO}) ~, & \text{if AoC Step's AoC Type = EOP and EstimateType is not A} \\
\text{Imported Actual} ~, & \text{otherwise}
\end{array}
\right.
$$

where the value is also function of the [Estimate Type](../DataModel/DataStructure#EstimateType) and [Amount Type](../DataModel/DataStructure#AmountType), and the $\text{Imported Actual}$ value is described [here]().

In [0]:
public interface ActualBase : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? AccidentYear), ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
            builder.ForScope<ActualBase>(s => s.WithApplicability<EmptyValuesActual>(x => x.GetStorage().ImportFormat == ImportFormats.Actual 
                                                                                       && !x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode)
                                                                                       && x.Identity.Id.AocType == AocTypes.AM)
                                               .WithApplicability<EndOfPeriodActual>(x => x.GetStorage().ImportFormat != ImportFormats.Cashflow 
                                                                                       && !x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode)
                                                                                       && x.Identity.Id.AocType == AocTypes.EOP 
                                                                                       && x.Identity.EstimateType != EstimateTypes.A)
                                       );
    public double Value => GetStorage().GetValue(Identity.Id, Identity.AmountType, Identity.EstimateType, Identity.AccidentYear); 
}

public interface EndOfPeriodActual : ActualBase
{
    double ActualBase.Value => GetScope<PreviousAocSteps>((Identity.Id, InputSource.Actual)).Values
                                        .Sum(aocStep => GetScope<ActualBase>((Identity.Id with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}, Identity.AmountType, Identity.EstimateType, Identity.AccidentYear)).Value);
}

public interface EmptyValuesActual : ActualBase
{
    double ActualBase.Value => 0;
}

## Actuals
The Actuals correspond to ActualBase values with estimate type $A$.
The only valid AoC Step is Release:

$$
\text{Actual} (\text{Release}) = \text{Actual Base} (\text{Release})|_{\text{Estimate Type} = A}
$$

In [0]:
public interface Actual : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.A;
    
    private int?[] accidentYears => GetStorage().GetAccidentYears(Identity.DataNode).ToArray();
    
    [NotVisible]
    ActualBase[] Actuals => GetScope<ValidAmountType>(Identity.DataNode).ActualAmountTypes
                                                            .SelectMany(at => accidentYears
                                                                .Select(ay => GetScope<ActualBase>((Identity, at, EstimateType, ay)))).ToArray();
}

## Advance Actuals

Advance Actuals are cash flows with due date inside the reporting period but occured *before* the reporting period - They include:
- Receivable Claims
- Receivable Expenses
- Payable Premiums

Advance Actuals are given by

$$
\text{Advance Actual} (\text{AoC step}) = \text{Actual Base} (\text{AoC step})|_{\text{Estimate Type} = AA}
$$

In [0]:
public interface AdvanceActual : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.AA;
    
    private int?[] accidentYears => GetStorage().GetAccidentYears(Identity.DataNode).ToArray();
    
    [NotVisible]
    ActualBase[] Actuals => GetScope<ValidAmountType>(Identity.DataNode).ActualAmountTypes
                                                            .SelectMany(at => accidentYears
                                                                .Select(ay => GetScope<ActualBase>((Identity, at, EstimateType, ay)))).ToArray();
}

## Overdue Actuals

Overdue Actuals are cash flows with due date inside the reporting period but occur *after* the reporting period - They contain:
- Payable Claims
- Payable Expenses
- Receivable Premiums

The sign convention is the inverse of the default behavior - In particular: Premiums have positive value, whereas Claims and Expenses have negative value.

The Overdue Actuals are given by

$$
\text{Overdue Actual} (\text{AoC step}) = \text{Actual Base} (\text{AoC step})|_{\text{Estimate Type} = OA}
$$

In [0]:
public interface OverdueActual : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.OA;
    
    private int?[] accidentYears => GetStorage().GetAccidentYears(Identity.DataNode).ToArray();
    
    [NotVisible]
    ActualBase[] Actuals => GetScope<ValidAmountType>(Identity.DataNode).ActualAmountTypes
                                                            .SelectMany(at => accidentYears
                                                                .Select(ay => GetScope<ActualBase>((Identity, at, EstimateType, ay)))).ToArray();
}

## Deferrable Actuals

Finally, Deferrable Actuals are given by

$$
\text{Deferrable Actual} (\text{AoC step}) = \left\{
\begin{array}{rl}
\sum_{\text{Amount Type}~\in~\text{\{ACA, AEA}\}}\text{Actual Base}(\rm{CF})|_{\text{Estimate Type = A}} ~, & \text{if AoC Step's AoC Type = CF } \\
- \text{Amortization Factor} \cdot \big(  \text{Deferrable Actual}(\rm{BOP}) + \text{Deferrable Actual}(\rm{CF}) \big) ~, & \text{if AoC Step's AoC Type = AM } \\
\text{Deferrable Actual}(\rm{BOP}) + \text{Deferrable Actual}(\rm{CF}) + \text{Deferrable Actual}(\rm{AM}) ~, & \text{if AoC Step's AoC Type = EOP } \\
\text{Input Actual}|_{\text{Estimate Type = DA}} ~, & \text{ otherwise } \\
\end{array}
\right.
$$

where $ACA$ and $AEA$ are *Aquisition* Amount Types from **Attributable Commission** and **Attributable Expenses**, respectively.

In [0]:
public interface DeferrableActual : IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
            builder.ForScope<DeferrableActual>(s => s.WithApplicability<DeferrableActualForCurrentBasis>(x => x.Identity.ValuationApproach == ValuationApproaches.VFA, 
                                                                                                              p => p.ForMember(s => s.EconomicBasis))
                                                     .WithApplicability<ReleaseDeferrable>(x => x.Identity.AocType == AocTypes.CF)
                                                     .WithApplicability<AmortizationDeferrable>(x => x.Identity.AocType == AocTypes.AM)
                                                     .WithApplicability<EndOfPeriodDeferrable>(x => x.Identity.AocType == AocTypes.EOP)
                                       );
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.DA;
    
    [NotVisible] string EconomicBasis => EconomicBases.L;
    
    public double Value => GetStorage().GetValue(Identity, (string)null, EstimateType, (int?)null);
}

public interface DeferrableActualForCurrentBasis : DeferrableActual
{
    [NotVisible] string DeferrableActual.EconomicBasis => EconomicBases.C;
}

public interface ReleaseDeferrable : DeferrableActual
{
    double DeferrableActual.Value => GetStorage().GetAttributableExpenseAndCommissionAmountType().Sum(at => GetScope<ActualBase>((Identity, at, EstimateTypes.A, (int?)null)).Value);
}

public interface AmortizationDeferrable : DeferrableActual
{
    private double AmortizationFactor => GetScope<CurrentPeriodAmortizationFactor>(Identity, o => o.WithContext(EconomicBasis)).Value;
    private double AggregatedDeferrable => GetScope<PreviousAocSteps>((Identity, InputSource.Actual)).Values
                                            .Sum(aocStep => GetScope<DeferrableActual>(Identity with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}).Value);
    double DeferrableActual.Value => -1d * AggregatedDeferrable * AmortizationFactor;
}

public interface EndOfPeriodDeferrable : DeferrableActual
{
    double DeferrableActual.Value => GetScope<PreviousAocSteps>((Identity, InputSource.Actual)).Values
                                        .Sum(aocStep => GetScope<DeferrableActual>(Identity with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}).Value);
}